In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interbank20/sample_submission.csv
/kaggle/input/interbank20/y_train.csv
/kaggle/input/interbank20/sunat_test.csv
/kaggle/input/interbank20/rcc_test.csv
/kaggle/input/interbank20/se_test.csv
/kaggle/input/interbank20/sunat_train.csv
/kaggle/input/interbank20/se_train.csv
/kaggle/input/interbank20/censo_test.csv
/kaggle/input/interbank20/productos.csv
/kaggle/input/interbank20/censo_train.csv
/kaggle/input/interbank20/rcc_train.csv


In [2]:
import pandas as pd
from tqdm.notebook import tqdm 
import gc
import numpy as np

In [3]:
import statistics as stat
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

## Load Bases

In [4]:
sunat_train = pd.read_csv('/kaggle/input/interbank20/sunat_train.csv')
rcc_train = pd.read_csv('/kaggle/input/interbank20/rcc_train.csv')
se_train = pd.read_csv('/kaggle/input/interbank20/se_train.csv', index_col = 'key_value')
y_train = pd.read_csv('/kaggle/input/interbank20/y_train.csv', index_col = 'key_value')

sunat_test = pd.read_csv('/kaggle/input/interbank20/sunat_test.csv')
rcc_test = pd.read_csv('/kaggle/input/interbank20/rcc_test.csv')
se_test = pd.read_csv('/kaggle/input/interbank20/se_test.csv', index_col = 'key_value')

## Procesar RCC

In [5]:
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [7]:
dict_ = {'codmes': 'int32',
 'key_value': 'int32',
 'condicion': 'int32',
 'tipo_credito': 'int32',
 'cod_instit_financiera': 'int32',
 'PRODUCTO': 'int32',
 'RIESGO_DIRECTO': 'int32',
 'COD_CLASIFICACION_DEUDOR': 'int32'}
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [8]:
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'condicion':['max','min','mean','std','sum'],
           'saldo':['max','min','mean','std','sum'],
           'cod_instit_financiera':['nunique','min','max',moda],
           'PRODUCTO':['nunique','min','max',moda],
           'COD_CLASIFICACION_DEUDOR':['nunique','max','min','sum','mean','std',moda],
           'RIESGO_DIRECTO':['nunique','max','min','sum','mean','std',moda],
           'tipo_credito':['nunique','max','min','sum','mean','std',moda]
          }

In [12]:
list_rcc_train_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()

        print(f'haciendo aggregate de {i}')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201802
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201801
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201712
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201711
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201710
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo 

In [13]:
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        print(f'haciendo aggregate de {i}')
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201902
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201901
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201812
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201811
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo aggregate de 201810
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo 

In [14]:
### asegurar que las columnas esten en ambas bases (train/test)
print(rcc_train_.shape, rcc_test_.shape)
keep_columns = list(set(rcc_train_.columns).intersection(rcc_test_.columns))
rcc_train_ = rcc_train_[keep_columns].copy()
rcc_test_ = rcc_test_[keep_columns].copy()
print(rcc_train_.shape, rcc_test_.shape)

(358487, 539) (396666, 553)
(358487, 539) (396666, 539)


In [15]:
### imputar datos categoricos
var_categ = [i for i in rcc_train_.columns if 'mode' in i]
for i in var_categ:
    rcc_train_[i] = rcc_train_[i].fillna(rcc_train_[i].value_counts().index[0])
    rcc_test_[i] = rcc_test_[i].fillna(rcc_train_[i].value_counts().index[0])

In [16]:
### mean encoding
rcc_train_ = pd.concat([rcc_train_, y_train], axis=1)
for i in var_categ:
    me = rcc_train_.groupby([i])['target'].mean().to_dict() 
    rcc_train_[f'{i}_me'] =  rcc_train_[i].map(me) 
    rcc_test_[f'{i}_me'] =  rcc_test_[i].map(me)
    ### imputar datos faltantes
    rcc_test_[f'{i}_me'] =  rcc_test_[f'{i}_me'].fillna(rcc_train_[f'{i}_me'].median())
    
    
rcc_train_.drop('target', axis=1, inplace=True)
rcc_train_.drop(var_categ, axis=1, inplace=True)
rcc_test_.drop(var_categ, axis=1, inplace=True)

In [17]:
### unir RCC en la base final
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

## Procesar SE

In [18]:
dict_ = {'sexo':'int32',
         'est_cvl':'int32',
         'sit_lab':'int32',
         'cod_ocu':'int32',
         'ctd_hijos':'int32',
         'flg_sin_email':'int32',
         'ctd_veh':'int32',
         'lgr_vot':'int32',
         'prv':'int32',
         'dto':'int32',
         'rgn':'int32',
         'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [19]:
### imputar datos faltantes en la base se_[train/test]
se_train['edad'].fillna(se_train['edad'].median(), inplace=True)
se_test['edad'].fillna(se_train['edad'].median(), inplace=True)
se_train['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)
se_test['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)

In [20]:
### mean encoding
var_categ=['sit_lab','lgr_vot','sexo','rgn','tip_lvledu','flg_sin_email','prv','est_cvl','cod_ocu','dto']
# var_caunti=['edad','ctd_hijos','ctd_veh','cod_ubi']
se_train['target'] = y_train.loc[se_train.index]
for i in var_categ:
    me = se_train.groupby([i])['target'].mean().to_dict() 
    se_train[f'{i}_me'] =  se_train[i].map(me) 
    se_test[f'{i}_me'] =  se_test[i].map(me)
    ### imputar datos faltantes
    se_test[f'{i}_me'] =  se_test[f'{i}_me'].fillna(se_train[f'{i}_me'].median())
se_train.drop('target', axis=1, inplace=True)
se_train.drop(var_categ, axis=1, inplace=True)
se_test.drop(var_categ, axis=1, inplace=True)

In [21]:
### unir SE en la base final
train = train.join(se_train) 
test = test.join(se_test)
del se_train, se_test

In [22]:
### imputar valores faltantes
var_caunti = ['lgr_vot_me','sit_lab_me','cod_ubi','tip_lvledu_me','prv_me','rgn_me',
 'ctd_hijos','edad','cod_ocu_me','ctd_veh','est_cvl_me','dto_me','flg_sin_email_me','sexo_me']
for i in var_caunti:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())

## Procesar SUNAT

In [23]:
### eliminar registros duplicados
sunat_train.drop_duplicates(inplace=True)
sunat_test.drop_duplicates(inplace=True)
sunat_train.shape, sunat_test.shape

((292479, 18), (318821, 18))

In [24]:
#### eliminando registros con fecalta y fecbaja con valores nulos
mask = ~((sunat_train['fecalta'].astype('str')=='nan') & (sunat_train['fecbaja'].astype('str')=='nan'))
sunat_train = sunat_train[mask].copy()
mask = ~((sunat_test['fecalta'].astype('str')=='nan') & (sunat_test['fecbaja'].astype('str')=='nan'))
sunat_test = sunat_test[mask].copy()
sunat_train.shape, sunat_test.shape

((272028, 18), (293080, 18))

In [25]:
dict_ = {'tipcontribuyente': 'int32',
         'tippersona': 'int32',
         'ciiu': 'int32',
         'ubigeo': 'int32',
         'condiciondomicilio': 'int32',
         'estadocontribuyente': 'int32',
         'codvia': 'int32',
         'codzona': 'int32',
         'contabilidad': 'int32',
         'facturacion': 'int32',
         'domiciliado': 'int32',
         'comercioexterior': 'int32',
         'cargorele': 'int32',
         'codentidadtributo': 'int32',
         'estadotributo': 'int32'}
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [26]:
sunat_train['diff_fech'] = sunat_train['fecbaja'] - sunat_train['fecalta']
sunat_test['diff_fech'] = sunat_test['fecbaja'] - sunat_test['fecalta']

In [27]:
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
           'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['mean','max', 'nunique'],
           'fecbaja':['mean','max', 'nunique'], 
           'diff_fech':['mean','max', 'nunique'], 
            }

sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
del sunat_train, sunat_test

In [ ]:
# var_categ = [i for i in sunat_train_.columns if 'mode' in i]
# for i in var_categ:
#     sunat_train_[i] = sunat_train_[i].fillna(sunat_train_[i].value_counts().index[0])
#     sunat_test_[i] = sunat_test_[i].fillna(sunat_train_[i].value_counts().index[0])

In [ ]:
# sunat_train_['target'] = y_train.loc[sunat_train_.index]
# for i in var_categ:
#     me = sunat_train_.groupby([i])['target'].mean().to_dict() 
#     sunat_train_[f'{i}_me'] =  sunat_train_[i].map(me) 
#     sunat_test_[f'{i}_me'] =  sunat_test_[i].map(me)
#     ### imputar datos faltantes
#     sunat_test_[f'{i}_me'] =  sunat_test_[f'{i}_me'].fillna(sunat_train_[f'{i}_me'].median())
# sunat_train_.drop('target', axis=1, inplace=True)
# sunat_train_.drop(var_categ, axis=1, inplace=True)
# sunat_test_.drop(var_categ, axis=1, inplace=True)

In [28]:
### unir SUNAT en la base final
train = train.join(sunat_train_)
test = test.join(sunat_test_)
del sunat_train_, sunat_test_

In [29]:
test.isna().sum().sort_values().tail(40)/len(test)

saldo_std_ult1mes              0.003855
contabilidad_nunique           0.422910
fecbaja_nunique                0.422910
fecalta_nunique                0.422910
fecalta_max                    0.422910
fecalta_mean                   0.422910
estadotributo_nunique          0.422910
codentidadtributo_nunique      0.422910
cargorele_nunique              0.422910
comercioexterior_nunique       0.422910
domiciliado_nunique            0.422910
facturacion_nunique            0.422910
codzona_nunique                0.422910
ubigeo_nunique                 0.422910
diff_fech_nunique              0.422910
tipcontribuyente_nunique       0.422910
estadocontribuyente_nunique    0.422910
condiciondomicilio_nunique     0.422910
tippersona_nunique             0.422910
codvia_nunique                 0.422910
ciiu_nunique                   0.422910
condiciondomicilio_mode        0.431998
estadotributo_mode             0.441535
codentidadtributo_mode         0.444432
facturacion_mode               0.444452


## Training

In [30]:
test.isna().sum().sort_values()

tipo_credito_8_saldo_ult4meses                     0
tipo_credito_sum_ult12mes                          0
tipo_credito_max_ult5mes                           0
COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses         0
tipo_credito_6_condicion_ult3meses                 0
                                               ...  
tipcontribuyente_mode                         188733
diff_fech_max                                 357149
fecbaja_mean                                  357149
fecbaja_max                                   357149
diff_fech_mean                                357149
Length: 592, dtype: int64

In [31]:
train.shape, test.shape

((358487, 592), (396666, 592))

In [32]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]

    learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', max_depth = 6)
    learner.fit(Xt, yt,  early_stopping_rounds=10, 
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
    gc.collect()
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))

********** 0 **********
Learning rate set to 0.128212
0:	test: 0.7395598	test1: 0.7419765	best: 0.7419765 (0)	total: 473ms	remaining: 7m 52s
50:	test: 0.8318550	test1: 0.8305138	best: 0.8305138 (50)	total: 21.9s	remaining: 6m 47s
100:	test: 0.8412418	test1: 0.8382830	best: 0.8382830 (100)	total: 43.2s	remaining: 6m 24s
150:	test: 0.8478826	test1: 0.8429005	best: 0.8429005 (150)	total: 1m 4s	remaining: 6m 1s
200:	test: 0.8525536	test1: 0.8455408	best: 0.8455408 (200)	total: 1m 24s	remaining: 5m 34s
250:	test: 0.8563702	test1: 0.8473145	best: 0.8473145 (250)	total: 1m 43s	remaining: 5m 10s
300:	test: 0.8597665	test1: 0.8484027	best: 0.8484198 (296)	total: 2m 3s	remaining: 4m 47s
350:	test: 0.8627378	test1: 0.8491692	best: 0.8491692 (350)	total: 2m 24s	remaining: 4m 27s
400:	test: 0.8652489	test1: 0.8495482	best: 0.8495839 (393)	total: 2m 44s	remaining: 4m 5s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.8495838977
bestIteration = 393

Shrink model to first 394 itera

In [34]:
fi.sort_values(ascending=False).head(50)

dto_me                                            0.049566
COD_CLASIFICACION_DEUDOR_mean_ult1mes             0.036564
tipo_credito_sum_ult12mes                         0.027809
tip_lvledu_me                                     0.023765
cod_instit_financiera_mode_ult1mes_me             0.023747
COD_CLASIFICACION_DEUDOR_max_ult1mes              0.019567
tipo_credito_11_saldo_ult1meses                   0.018223
edad                                              0.016759
RIESGO_DIRECTO_1_saldo_ult1meses                  0.016338
RIESGO_DIRECTO_sum_ult12mes                       0.016164
COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.014326
PRODUCTO_min_ult1mes                              0.012994
sexo_me                                           0.012839
flg_sin_email_me                                  0.012453
cod_instit_financiera_max_ult1mes                 0.011614
RIESGO_DIRECTO_-1_saldo_ult1meses                 0.011426
saldo_sum_ult1mes                                 0.0109

In [35]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]

    learner = LGBMClassifier(n_estimators=1000, max_depth = 6, boosting_type='gbdt', min_child_samples=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))  

********** 0 **********


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.841785	training's binary_logloss: 0.304211	valid_1's auc: 0.835352	valid_1's binary_logloss: 0.309133
[100]	training's auc: 0.853658	training's binary_logloss: 0.294776	valid_1's auc: 0.844119	valid_1's binary_logloss: 0.30244
[150]	training's auc: 0.860547	training's binary_logloss: 0.289294	valid_1's auc: 0.847694	valid_1's binary_logloss: 0.299614
[200]	training's auc: 0.865265	training's binary_logloss: 0.285425	valid_1's auc: 0.849369	valid_1's binary_logloss: 0.298155
[250]	training's auc: 0.869851	training's binary_logloss: 0.281831	valid_1's auc: 0.850645	valid_1's binary_logloss: 0.29717
[300]	training's auc: 0.872976	training's binary_logloss: 0.279267	valid_1's auc: 0.851295	valid_1's binary_logloss: 0.296619
[350]	training's auc: 0.87605	training's binary_logloss: 0.276747	valid_1's auc: 0.851701	valid_1's binary_logloss: 0.296252
[400]	training's auc: 0.878943	training's binary_logloss: 0.2

In [36]:
fi.sort_values(ascending=False).head(50)

dto_me                                            0.025456
tipo_credito_sum_ult12mes                         0.021162
edad                                              0.019146
RIESGO_DIRECTO_1_saldo_ult1meses                  0.016159
cod_ubi                                           0.015293
tipo_credito_sum_ult1mes                          0.014216
RIESGO_DIRECTO_sum_ult12mes                       0.013894
RIESGO_DIRECTO_1_saldo_ult12meses                 0.013583
RIESGO_DIRECTO_-1_saldo_ult1meses                 0.012958
cod_instit_financiera_max_ult1mes                 0.012639
saldo_mean_ult12mes                               0.012207
cod_instit_financiera_mode_ult1mes_me             0.012124
cod_instit_financiera_mode_ult12mes_me            0.012057
tipo_credito_11_saldo_ult1meses                   0.011964
RIESGO_DIRECTO_mean_ult1mes                       0.011203
tip_lvledu_me                                     0.010972
saldo_min_ult1mes                                 0.0108

In [37]:
from catboost import CatBoostClassifier
learner_catboost = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 6)
learner_catboost.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

Learning rate set to 0.126968
0:	total: 420ms	remaining: 6m 59s
50:	total: 25.2s	remaining: 7m 48s
100:	total: 49.5s	remaining: 7m 21s
150:	total: 1m 12s	remaining: 6m 49s
200:	total: 1m 35s	remaining: 6m 19s
250:	total: 1m 59s	remaining: 5m 55s
300:	total: 2m 21s	remaining: 5m 29s
350:	total: 2m 45s	remaining: 5m 5s
400:	total: 3m 8s	remaining: 4m 41s
450:	total: 3m 31s	remaining: 4m 17s
500:	total: 3m 58s	remaining: 3m 57s
550:	total: 4m 21s	remaining: 3m 33s
600:	total: 4m 45s	remaining: 3m 9s
650:	total: 5m 7s	remaining: 2m 45s
700:	total: 5m 30s	remaining: 2m 20s
750:	total: 5m 53s	remaining: 1m 57s
800:	total: 6m 16s	remaining: 1m 33s
850:	total: 6m 39s	remaining: 1m 9s
900:	total: 7m 1s	remaining: 46.3s
950:	total: 7m 24s	remaining: 22.9s
999:	total: 7m 48s	remaining: 0us


In [38]:
from lightgbm import LGBMClassifier
learner_lightgbm = LGBMClassifier(n_estimators=1000, max_depth=6, boosting_type='gbdt', min_child_samples=1000)
learner_lightgbm.fit(train, y_train, eval_metric="auc", verbose=True)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LGBMClassifier(max_depth=6, min_child_samples=1000, n_estimators=1000)

In [47]:
test_probs_catboost = pd.Series(learner_catboost.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_lightgbm = pd.Series(learner_lightgbm.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_mean = test_probs_catboost*0.4 + test_probs_lightgbm*0.6
test_probs_mean.name = 'target'
test_probs_mean.shape

(396666,)

In [48]:
test_probs_mean.to_csv("test65.csv")

![](http://)<a href='test65.csv'>Download csv</a>